In [25]:
from re import findall

import pandas as pd
from requests import get

import constants
import utils

## TODO: Take the sheet directly from Google Sheets instead of CSV

In [26]:
pl_export = utils.load_dataframe(constants.OWNED_PLAYERS_FILENAME)

In [27]:
owned_players = utils.clean_pl_export(pl_export)

In [28]:
single_teams = []
for column in owned_players.columns:
    if column == 'Team Position':
        continue
    single_team = owned_players[['Team Position', column]].copy()
    single_team.columns = ['Team Position', 'Player']
    single_team['PL Owner'] = column
    single_teams.append(single_team)
len(single_teams)

10

In [29]:
owned_players_full = pd.concat(single_teams)
# Remove empty bench spots
owned_players_full = owned_players_full.loc[~pd.isna(owned_players_full['Player'])]
owned_players_full

,Team Position,Player,PL Owner
1,C,JTRealmuto-CPhi,Washington Generals
3,1B,PGoldschmidt-1BSTL,Washington Generals
5,2B,BStott-2BPhi,Washington Generals
7,3B,ARiley-3BAtl,Washington Generals
9,SS,FLindor-SSNYM,Washington Generals
...,...,...,...
44,BN,KSenga-SPNYM,Nuclear Arms
45,BN,DWilliams-RPMil,Nuclear Arms
46,BN,BRooker-OFOak,Nuclear Arms
47,BN,GCleavinger-RPTam,Nuclear Arms


In [30]:
owned_players_full[['PL Name', 'Pos Team']] = owned_players_full['Player'].str.rsplit('-', n=1, expand=True)
owned_players_full['Player Positions'] = owned_players_full['Pos Team'].str.strip().str[:-3]
owned_players_full['Team'] = owned_players_full['Pos Team'].str.strip().str[-3:].str.upper()
owned_players_full

,Team Position,Player,PL Owner,PL Name,Pos Team,Player Positions,Team
1,C,JTRealmuto-CPhi,Washington Generals,JTRealmuto,CPhi,C,PHI
3,1B,PGoldschmidt-1BSTL,Washington Generals,PGoldschmidt,1BSTL,1B,STL
5,2B,BStott-2BPhi,Washington Generals,BStott,2BPhi,2B,PHI
7,3B,ARiley-3BAtl,Washington Generals,ARiley,3BAtl,3B,ATL
9,SS,FLindor-SSNYM,Washington Generals,FLindor,SSNYM,SS,NYM
...,...,...,...,...,...,...,...
44,BN,KSenga-SPNYM,Nuclear Arms,KSenga,SPNYM,SP,NYM
45,BN,DWilliams-RPMil,Nuclear Arms,DWilliams,RPMil,RP,MIL
46,BN,BRooker-OFOak,Nuclear Arms,BRooker,OFOak,OF,OAK
47,BN,GCleavinger-RPTam,Nuclear Arms,GCleavinger,RPTam,RP,TAM


In [31]:
def get_player_id_from_pl_name(pl_name, pl_team):
    player_id = -1
    search_value = None

    # Split the PL-formatted name (JTRealmuto) on capital letters. Take the last value (name) unless special handling is required.
    split_name = findall('[A-Z][^A-Z]*', pl_name)
    first_initial = split_name[0][0]
    split_last_name_index = -1
    last_name = split_name[split_last_name_index]

    # Handle special cases...
    #if pl_name in constants.PLAYER_LOOKUP_VALUES.keys():
    #    search_value = constants.PLAYER_LOOKUP_VALUES[pl_name]
    search_value = last_name
    while search_value.upper() in ('JR', 'I'):
        split_last_name_index -= 1
        search_value = split_name[split_last_name_index]

    # Catch Mc* or O'* names
    if split_name[split_last_name_index - 1].upper() in ('MC', "O'"):
        search_value = split_name[split_last_name_index - 1] + search_value

    # Lookup player in statsapi
    lookup_results = get(f'{constants.STATSAPI_PEOPLE_SEARCH_PATH}?names={search_value}&hydrate=currentTeam', timeout=30).json()['people']

    print(f'Got {len(lookup_results)} results from the API for player {pl_name}, team {pl_team}, search value {search_value}')

    for player in lookup_results:
        team_id = utils.get_team_id(pl_team)
        if first_initial == player['fullName'][0] and team_id == player['currentTeam']['id']:
            player_id = player['id']
            break

    if player_id == -1:
        print(f'Could not match PL player {pl_name} to an ID, lookup results: {lookup_results}')

    return player_id

In [32]:
owned_players_full['Player ID'] = owned_players_full.apply(lambda row: utils.get_player_id_from_pl_name(row['PL Name'], row['Team']), axis=1)
owned_players_full

Got 1 results from the API for player JTRealmuto, team PHI, search value Realmuto
Got 1 results from the API for player PGoldschmidt, team STL, search value Goldschmidt
Got 4 results from the API for player BStott, team PHI, search value Stott
Got 23 results from the API for player ARiley, team ATL, search value Riley
Got 1 results from the API for player FLindor, team NYM, search value Lindor
Got 1 results from the API for player ABregman, team HOU, search value Bregman
Got 3 results from the API for player BBichette, team TOR, search value Bichette
Got 3 results from the API for player RAcuñaJr, team ATL, search value Acuña
Got 6 results from the API for player GSpringer, team TOR, search value Springer
Got 1 results from the API for player BNimmo, team NYM, search value Nimmo
Got 2 results from the API for player LGurrielJr, team ARI, search value Gurriel
Got 2 results from the API for player STorkelson, team DET, search value Torkelson
Got 2 results from the API for player JCandela

,Team Position,Player,PL Owner,PL Name,Pos Team,Player Positions,Team,Player ID
1,C,JTRealmuto-CPhi,Washington Generals,JTRealmuto,CPhi,C,PHI,592663
3,1B,PGoldschmidt-1BSTL,Washington Generals,PGoldschmidt,1BSTL,1B,STL,502671
5,2B,BStott-2BPhi,Washington Generals,BStott,2BPhi,2B,PHI,681082
7,3B,ARiley-3BAtl,Washington Generals,ARiley,3BAtl,3B,ATL,663586
9,SS,FLindor-SSNYM,Washington Generals,FLindor,SSNYM,SS,NYM,596019
...,...,...,...,...,...,...,...,...
44,BN,KSenga-SPNYM,Nuclear Arms,KSenga,SPNYM,SP,NYM,673540
45,BN,DWilliams-RPMil,Nuclear Arms,DWilliams,RPMil,RP,MIL,642207
46,BN,BRooker-OFOak,Nuclear Arms,BRooker,OFOak,OF,OAK,667670
47,BN,GCleavinger-RPTam,Nuclear Arms,GCleavinger,RPTam,RP,TAM,664076


In [33]:
utils.save_dataframe(owned_players_full, constants.OWNED_PLAYERS_IDS_FILENAME)